Day 36 : 21.03.10

# Approaching (Almost) Any NLP Problem on Kaggle

https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

여기선 자연어 처리 문제에 대해 이야기하고자 한다. 우선 기본적인 첫 번째 모델을 만든 후 다른 변수를 사용해 개선한다. 또한 신경망이 얼마나 심층적으로 사용될 수 있는지 보고 일반적으로 앙상블에 대한 아이디어로 끝낼 것이다.

여기서 다룰 문제:
- tf-idf
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [4]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [6]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


이 문제는 저자를 예측하는 문제다. 즉, text에 주어진 EAP, HPL, MWS를 예측해야한다. 간단히 말해서 text 분류는 3가지 클래스로 이루어진다.

이 특정 문제에 대해 Kaggle은 multi-class log-loss(다중 클래스 로그 손실)을 평가 지표로 지정했다.

In [7]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """ 로그 손실 메트릭의 다중 클래스 버전 """
    # actual : 실제 target 클래스를 포함하는 배열
    # predicted : 클래스 예측이 있는 행렬, 클래스당 하나의 확률
    
    # 이진 배열이 아닌 actual을 이진 배열로 변환
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i,val] = 1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)  ### np.clip(array, min, max) : array의 min, max 범위 넘어가는 요소를 작으면 min, 크면 max로 변환
    rows = actual.shape[0]
    vsota = np.sum(actual*np.log(clip))
    
    return -1.0 / rows*vsota

scikit-learn의 LabelEncoder를 사용해 text 레이블을 정수로 변환 : 0,1,2

In [8]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

더 나아가기 전에 데이터를 training, validation 셋으로 나누는 것이 중요하다. scikit-learn의 model_selection 모듈에서 train_test_split을 사용한다.

In [10]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values,y,
                                                 stratify=y, random_state=42, test_size=0.1, shuffle=True)

In [11]:
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


## Building Basic Models

첫 번재 모델을 만들어보자.

첫 모델을 단순 TF-IDF(Term Frequency - Inverse Document Frequency) 후 단순 로지스틱 회귀분석이다.

In [19]:
# 항상 이 매개변수들로 해라. 거의 항상 이걸로 작동된다.
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                     token_pattern=r'\w{1,}', ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')
# training, test 셋에 TF-IDF 적합 (준지도학습)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [45]:
# TF-IDF에 단순 로지스틱 회귀분석 적합
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

AttributeError: 'str' object has no attribute 'decode'

0.626의 다중 클래스 로그 손실을 갖는 첫 모델을 생성했다. 하지만 더 좋은 점수를 위해 다른 데이터로 동일한 모델을 살펴보자.

TF-IDF를 사용하는 대신 단어 수를 변수로 사용할 수도 있다. 이 작업은 scikit-learn에서 CountVectorizer를 사용해 쉽게 할 수 있다.

In [26]:
ctv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')

# training, test 셋에 Count Vectorizer 적합 (준지도학습)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [27]:
# Counts에 단순 로지스틱 회귀 적합
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

AttributeError: 'str' object has no attribute 'decode'

첫 모델을 0.1점 향상시켰다!

다음으로, Naive Bayes 모델을 시도해보자. 다음 두 데이터셋에서 Naive bayes를 사용할 때 어떤 일이 일어나는지 살펴보자.

In [28]:
# TF-IDF에 단순 Naive Bayes 적합
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.578


좋은 성능이지만 count에 대한 로지스틱회귀분석이 더 좋다. 대신 count 데이터에 이 모델을 사용하면 어떻게 될까?

In [29]:
# Counts에 단순 Naive Bayes 적합
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.485


다음은 SVM을 적용해보자. SVM은 시간이 많이 걸리므로 적용하기 전 Singlular Value Decomposition을 사용해 TF-IDF의 변수 개수를 줄인다. 또한 SVM을 적용하기 전 데이터를 표준화해야한다.

In [30]:
# 120개 요소를 선택해 SVD 적용. 120-200 요소가 SVM 모델에 충분하다
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# SVD로 얻은 데이터를 스케일링. 스케일링 없이 재사용할 변수 이름 재서렂ㅇ
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [31]:
# 단순 SVM 적합
clf = SVC(C=1.0, probability=True)  # 확률이 필요하기 때문
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.727


SVM이 이 데이터에서 성능이 좋지 않은 것 같다. 캐글에서 가장 유명한 알고리즘인 xgboost를 적용해보자.

In [32]:
# tf-idf에 xgboost 적합
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)  ### tocsc : 희소행렬로 만들기
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

c:\anaconda3\envs\study\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:29:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.781


In [35]:
import warnings
warnings.filterwarnings('ignore')
# tf-idf에 xgboost 적합
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:32:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.772


In [36]:
# tf-idf svd 변수에 xgboost 적용
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:36:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.768


In [37]:
# tf-idf svd 변수에 xgboost 적합
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:42:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.786


XGBoost는 안 좋아보인다. 하지만 정확하지 않다. 아직 매개변수 최적화를 하지 않았다. 최적화 방법은 다음 섹션에서 설명한다.

### Grid Search

매개변수를 최적화하기 위한 기술이다. 그리 효과적이지 않지만 사용할 그리드를 알고 있으면 좋은 결과를 얻을 수 있다. 

이 섹션에서 로지스틱 회귀분석을 사용한 Grid Search에 대해 설명한다. Grid Search를 시작하기 전, 점수 계산 함수를 만들어야한다. 이는 scikit-learn의 make_scorer 함수를 사용한다.

In [38]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

다음은 파이프라인이 필요하다. 여기서 시연하기 위해, SVD, 스케일링, 로지스틱 회귀분석으로 구성된 파이프라인을 사용한다. 하나의 모듈보다 더 많은 모듈을 파이프라인에 배치하는 것이 이해하기 좋다.

In [39]:
# SVD 초기화
svd = TruncatedSVD()
# standard scaler 초기화
scl = preprocessing.StandardScaler()
# 로지스틱 회귀분석
lr_model = LogisticRegression()
# 파이프라인 생성
clf = pipeline.Pipeline([('svd',svd), ('scl',scl), ('lr',lr_model)])

다음으로 매개변수의 그리드가 필요하다.

In [40]:
param_grid = {'svd__n_components':[120,180],
             'lr__C':[0.1,1.0,10],
             'lr__penalty':['l1','l2']}

따라서 SVD의 경우 120개와 180개의 요소를 평가하고, 로지스틱회귀분석의 경우 lr과 l2 패널티로 C의 세 가지 다른 값을 평가한다. 이제 이러한 매개 변수에 대한 Grid Search를 할 수 있다.

In [41]:
# Grid Search 모델 초기화
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=1, iid=True, refit=True, cv=2)

# Grid Search 모델 적합
model.fit(xtrain_tfv, ytrain)  # 여기선 전체 데이터를 사용할 수 있지만 xtrain만 사용한다
print('Best score : %0.3f' %model.best_score_)
print('Best parameters set :')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s: %r' %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.8s
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=nan, total=   2.4s
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................


...

[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.5s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   23.2s remaining:    0.0s


[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.1s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=nan, total=   2.0s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=nan, total=   2.0s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.775, total=   1.7s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.766, total=   1.6s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=180, score=-0.745, total=   2.2s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=180, score=-0.743, total=   

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   48.5s finished


Best score : -0.737
Best parameters set :
	lr__C: 0.1
	lr__penalty: 'l2'
	svd__n_components: 180


점수는 SVM 점수와 비슷하다. 이 기술은 아래와 같이 xgboost 또는 multinomial naive bayes를 finetuning하는 데 사용할 수 있다. 여기서 tf-idf 데이터를 사용한다.

In [44]:
nb_model = MultinomialNB()
clf = pipeline.Pipeline([('nb',nb_model)])  # pipleine 생성
param_grid = {'nb__alpha':[0.001,0.01,0.1,1,10,100]}  # parameter grid

# Grid Search 모델 초기화
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=1, iid=True, refit=True, cv=2)
# Grid Search 모델 적합
model.fit(xtrain_tfv, ytrain)  # 전체 데이터 사용 가능하지만 xtrain만 사용
print('Best score : %0.3f' %model.best_score_)
print('Best parameters set :')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s : %r' %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] .................... nb__alpha=0.001, score=-0.620, total=   0.0s
[CV] nb__alpha=0.001 .................................................
[CV] .................... nb__alpha=0.001, score=-0.641, total=   0.0s
[CV] nb__alpha=0.01 ..................................................
[CV] ..................... nb__alpha=0.01, score=-0.511, total=   0.0s
[CV] nb__alpha=0.01 ..................................................
[CV] ..................... nb__alpha=0.01, score=-0.523, total=   0.0s
[CV] nb__alpha=0.1 ...................................................
[CV] ...................... nb__alpha=0.1, score=-0.489, total=   0.0s
[CV] nb__alpha=0.1 ...................................................
[CV] ...................... nb__alpha=0.1, score=-0.495, total=   0.0s
[CV] nb__alpha=1 .....................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s


[CV] ...................... nb__alpha=100, score=-1.067, total=   0.0s
[CV] nb__alpha=100 ...................................................
[CV] ...................... nb__alpha=100, score=-1.067, total=   0.0s
Best score : -0.492
Best parameters set :
	nb__alpha : 0.1


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


원래 naive bayes 점수보다 8% 향상되었다. 

NLP 문제에서는 일반적으로 단어 벡터를 살펴본다. 단어 벡터를 통해 데이터에 대한 통찰력을 얻을 수 있다. 자세히 알아보자.

## Word Vectors

문장 벡터를 만드는 방법, 그것에 기계학습 모델을 만드는 방법을 간단히 설명하자면, GloVe 벡터를 사용한다. http://www-nlp.stanford.edu/data/glove.840B.300d.zip 여기서 GloVe 를 다운받을 수 있다.

In [49]:
# GloVe 벡터를 딕셔너리로 불러오기
embeddings_index = {}
f = open('../input/glove.840B.300d.txt', encoding='UTF8')
for line in tqdm(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        f.__next__()
f.close()

print('Found %s word vectors.' %len(embeddings_index))

2195997it [06:30, 5622.61it/s]

Found 2195864 word vectors.


In [51]:
# 이 함수는 전체 문장을 위한 정규화된 벡터를 만든다.
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words :
        try :
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v**2).sum())

In [53]:
# trainig, validation 셋을 위해 위의 함수를 사용해 문장 벡터 생성
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

100%|████████████████████████████████████████████████████████████████████████████| 1958/1958 [00:00<00:00, 2367.68it/s]


In [54]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

glove 변수에서 xgboost의 성능을 보자.

In [55]:
# glove 변수에 xgboost 적합
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[09:59:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:59:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.726


In [56]:
# glove 변수에 xgboost 적합
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[10:04:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:04:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.688


간단한 매개변수 튜닝으로 GloVe 변수에서 xgboost 점수를 향상시킬 수 있다. 

## Deep Learning

하지만 지금은 딥러닝 시대다.  GloVe 변수에 LSTM와 단순 심층 신경망을 학습시킨다. 심층신경망을 먼저 살펴보자.

In [57]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [58]:
# 신경망의 레이블을 이진화해야한다.
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [59]:
# Sequential 신경망에 레이어 3개 생성
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# 모델 compile
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [60]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, epochs=5,
         verbose=1, validation_data=(xvalid_glove_scl, yvalid_enc))

Epoch 1/5
276/276 [==============================] - 5s 9ms/step - loss: 1.0475 - val_loss: 0.7178
Epoch 2/5
276/276 [==============================] - 2s 5ms/step - loss: 0.7135 - val_loss: 0.6808
Epoch 3/5
276/276 [==============================] - 2s 6ms/step - loss: 0.6231 - val_loss: 0.6687
Epoch 4/5
276/276 [==============================] - 1s 5ms/step - loss: 0.5892 - val_loss: 0.6589
Epoch 5/5
276/276 [==============================] - 2s 6ms/step - loss: 0.5455 - val_loss: 0.6512


더 나은 결과를 얻기 위해 신경망의 매개변수를 조정하고, 더 많은 레이어를 추가하고, dropout을 늘려야한다. 여기선 xgboost보다 구현과 실행이 빠르고 더 나은 결과를 얻을 수 있다는 것을 최적화 없이 보여주려고 한다. 

LSTM을 사용해 텍스트 데이터를 토큰화해야한다.

In [61]:
# keras tokenizer 사용
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# sequences에 0 패딩
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [63]:
# 데이터셋에 있는 단어에 대한 embedding matrix 생성
embedding_matrix = np.zeros((len(word_index)+1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|█████████████████████████████████████████████████████████████████████████| 25943/25943 [00:00<00:00, 45243.84it/s]


In [67]:
# glove embeddings와 두 dense 레이어를 갖춘 LSTM
model = Sequential()
model.add(Embedding(len(word_index)+1, 300,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [68]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100,
         verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Epoch 1/100
35/35 [==============================] - 85s 2s/step - loss: 1.0917 - val_loss: 0.9267
Epoch 2/100
35/35 [==============================] - 71s 2s/step - loss: 0.9304 - val_loss: 0.7624
Epoch 3/100
35/35 [==============================] - 85s 2s/step - loss: 0.8252 - val_loss: 0.7007
Epoch 4/100
35/35 [==============================] - 91s 3s/step - loss: 0.7887 - val_loss: 0.6844
Epoch 5/100
35/35 [==============================] - 74s 2s/step - loss: 0.7584 - val_loss: 0.6644
Epoch 6/100
35/35 [==============================] - 98s 3s/step - loss: 0.7390 - val_loss: 0.6717
Epoch 7/100
35/35 [==============================] - 71s 2s/step - loss: 0.7239 - val_loss: 0.6322
Epoch 8/100
35/35 [==============================] - 97s 3s/step - loss: 0.6977 - val_loss: 0.6224
Epoch 9/100
35/35 [==============================] - 71s 2s/step - loss: 0.6742 - val_loss: 0.6026
Epoch 10/100
35/35 [==============================] - 95s 3s/step - loss: 0.6702 - val_loss: 0.5934
Epoch 11/

Epoch 83/100
35/35 [==============================] - 93s 3s/step - loss: 0.2336 - val_loss: 0.5731
Epoch 84/100
35/35 [==============================] - 76s 2s/step - loss: 0.2209 - val_loss: 0.5622
Epoch 85/100
35/35 [==============================] - 98s 3s/step - loss: 0.2246 - val_loss: 0.5673
Epoch 86/100
35/35 [==============================] - 62s 2s/step - loss: 0.2249 - val_loss: 0.5631
Epoch 87/100
35/35 [==============================] - 88s 3s/step - loss: 0.2196 - val_loss: 0.5670
Epoch 88/100
35/35 [==============================] - 62s 2s/step - loss: 0.2243 - val_loss: 0.5737
Epoch 89/100
35/35 [==============================] - 87s 3s/step - loss: 0.2073 - val_loss: 0.5661
Epoch 90/100
35/35 [==============================] - 64s 2s/step - loss: 0.2212 - val_loss: 0.5431
Epoch 91/100
35/35 [==============================] - 87s 3s/step - loss: 0.2083 - val_loss: 0.5826
Epoch 92/100
35/35 [==============================] - 63s 2s/step - loss: 0.2042 - val_loss: 0.5942


이제 점수가 0.5점 미만이다. 최적에서 멈추지 않고 더 많은 epochs를 실행할 것이지만 최적 반복에서 멈추기 위해 조기멈춤을 사용할 수 있다. 조기중지는 모델을 다시 컴파일하면 된다.

In [73]:
# glove embedding과 두 dense 레이어를 갖는 LSTM
model = Sequential()
model.add(Embedding(len(word_index)+1, 300,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# 조기중지 callback을 사용해 모델 적합
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100,
         verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 65s 2s/step - loss: 1.0892 - val_loss: 0.9289
Epoch 2/100
35/35 [==============================] - 84s 2s/step - loss: 0.9232 - val_loss: 0.7476
Epoch 3/100
35/35 [==============================] - 66s 2s/step - loss: 0.8208 - val_loss: 0.7206
Epoch 4/100
35/35 [==============================] - 91s 3s/step - loss: 0.7787 - val_loss: 0.6945
Epoch 5/100
35/35 [==============================] - 66s 2s/step - loss: 0.7743 - val_loss: 0.7361
Epoch 6/100
35/35 [==============================] - 94s 3s/step - loss: 0.7578 - val_loss: 0.6738
Epoch 7/100
35/35 [==============================] - 62s 2s/step - loss: 0.7233 - val_loss: 0.6473
Epoch 8/100
35/35 [==============================] - 89s 3s/step - loss: 0.7133 - val_loss: 0.6434
Epoch 9/100
35/35 [==============================] - 70s 2s/step - loss: 0.6827 - val_loss: 0.6079
Epoch 10/100
35/35 [==============================] - 91s 3s/step - loss: 0.6723 - val_loss: 0.6065
Epoch 11/

한 가지 의문은, 왜 이렇게 많은 dropout을 사용하는지이다. dropout이 없거나 거의 없는 모델을 적합하면 과적합하기 때문이다.

양방향 LSTM이 더 나은 결과를 얻을 수 있는지 알아보자. keras로 쉽게 할 수 있다.

In [79]:
# glove embeddings와 두 dense 레이어를 갖는 양방향 LSTM
model = Sequential()
model.add(Embedding(len(word_index)+1, 300,
                   weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# 조기중기 callback으로 모델 적합
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100,
         verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 447s 13s/step - loss: 1.0818 - val_loss: 0.8282
Epoch 2/100
35/35 [==============================] - 467s 13s/step - loss: 0.8783 - val_loss: 0.7249
Epoch 3/100
35/35 [==============================] - 506s 15s/step - loss: 0.8042 - val_loss: 0.6955
Epoch 4/100
35/35 [==============================] - 531s 15s/step - loss: 0.7768 - val_loss: 0.6880
Epoch 5/100
35/35 [==============================] - 544s 15s/step - loss: 0.7551 - val_loss: 0.6776
Epoch 6/100
35/35 [==============================] - 559s 16s/step - loss: 0.7298 - val_loss: 0.6675
Epoch 7/100
35/35 [==============================] - 567s 16s/step - loss: 0.7132 - val_loss: 0.6319
Epoch 8/100
35/35 [==============================] - 479s 14s/step - loss: 0.6816 - val_loss: 0.6390
Epoch 9/100
35/35 [==============================] - 479s 14s/step - loss: 0.6681 - val_loss: 0.5889
Epoch 10/100
35/35 [==============================] - 520s 15s/step - loss: 0.6300 - val_lo

거의 다 왔다. GRU의 두 레이어를 시도해보자.

In [80]:
# glove embeddings와 두 dense 레이어를 가진 GRU
model = Sequential()
model.add(Embedding(len(word_index)+1, 300,
                     weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# 조기중기 callback으로 모델 적합
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 479s 13s/step - loss: 1.0947 - val_loss: 0.8931
Epoch 2/100
35/35 [==============================] - 423s 12s/step - loss: 0.9144 - val_loss: 0.7906
Epoch 3/100
35/35 [==============================] - 427s 12s/step - loss: 0.8345 - val_loss: 0.7673
Epoch 4/100
35/35 [==============================] - 441s 13s/step - loss: 0.7951 - val_loss: 0.7119
Epoch 5/100
35/35 [==============================] - 361s 10s/step - loss: 0.7870 - val_loss: 0.6882
Epoch 6/100
35/35 [==============================] - 396s 11s/step - loss: 0.7410 - val_loss: 0.6759
Epoch 7/100
35/35 [==============================] - 354s 10s/step - loss: 0.7081 - val_loss: 0.6445
Epoch 8/100
35/35 [==============================] - 362s 10s/step - loss: 0.7113 - val_loss: 0.6367
Epoch 9/100
35/35 [==============================] - 376s 11s/step - loss: 0.6745 - val_loss: 0.6278
Epoch 10/100
35/35 [==============================] - 357s 10s/step - loss: 0.6504 - val_lo

최적화 작업을 계속하면 성능이 계속 향상된다. stemming과 lemmatization은 시도할만한 가치가 있지만 지금은 생략한다.

이제 앙상블을 확인해보자.

## Ensembling

In [91]:
# 메인 앙상블 클래스다. 다음 셀에서 어떻게 사용하는지 확인해보자.
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
import os
import sys
import logging

logging.basicConfig(level=logging.DEBUG,
                   format = '[%(asctime)s] %(levelname)s %(message)s',
                   datefmt = '%H:%M:%S', stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification',
                 optimize=roc_auc_score, lower_is_better=False, save_path=None):
        # model_dict : 모델 딕셔너리
        # num_folds : 앙상블 위한 fold 수
        # task_type : 분류인지 회귀인지
        # optimizer : AUC, logloss 등 최적화함수. y_test와 y_pred 두 개의 인수가 있어야함
        # lower_is_better : 낮은 최적화 함수 값이 더 좋거나 높음
        # save_path : 생성된 예측과 모델 pickle을 dump할 경로, 혹은 None
        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path
        
        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None
        
    def fit(self, training_data, y, lentrain):
        # training_data : tabular 형식의 training 데이터
        # y : 이진, 다항, 회귀
        # reutrn : 예측에 사용될 모델
        self.training_data = training_data
        self.y = y
        
        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info('Found %d classes', self.num_classes)
            self.lbl_enc = preprocessing.LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)
        
        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((
                train_prediction_shape[0], train_prediction_shape[1]*len(self.model_dict[level])
            ))
        
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level-1]
                
            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info('Training Level %d Fold #%d. Model #%d', level, foldnum, model_num)
                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    
                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index, (model_num*self.num_classes):(model_num*self.num_classes)+self.num_classes] = temp_train_predictions
                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info('Level %d. Fold #%d. Model #%d. Validation Score = %f', level, foldnum, model_num, validation_score)
                    foldnum += 1
                
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
            logger.info('Level %d. Model #%d. Mean Score = %f. Std Dev = %f', level, model_num, avg_score, std_score)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, 'train_predictions_level_' + str(level) + '.csv'), index=False, header=None)
            
        return self.train_prediction_dict
        
    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((
                test_prediction_shape[0], test_prediction_shape[1]*len(self.model_dict[level])
            ))
        self.test_data = test_data

        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level-1]
                temp_test = self.test_prediction_dict[level-1]

            for model_num, model in enumerate(self.model_dict[level]):
                logger.info('Training Fulldata Level %d. Model #%d', level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)
                logger.info('Predicting Test Level %d. Model #%d', level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num*self.num_classes):(model_num*self.num_classes)+self.num_classes] = temp_test_predictions
                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, 'test_predictions_level_' + str(level) + '.csv'), index=False, header=None)

        return self.test_prediction_dict

In [92]:
# 앙상블의 모든 단계에 사용될 데이터 특정
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],
              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='data/')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

[23:12:35] INFO Found 3 classes
[23:12:35] INFO Training Level 0 Fold #1. Model #0


AttributeError: 'str' object has no attribute 'decode'

In [ ]:
# 오류 확인
multiclass_logloss(yvalid, preds[1])

따라서 앙상블 점수가 크게 향상되는 것을 볼 수 있다!